# 목차
## 서론
### 1. 2020년 급격히 증가하는 주식 거래
### 2. 주식 열풍 속 Y&Z 세대
## 본론
### 1. Analysis
#### (1) 데이터 전처리
#### (2) Y&Z 세대의 주식투자 특징
#### (3) Modeling
### 2. Insight & Suggestion
#### (1) Insight : 평온해진 주식 시장과 혈기왕성한 Y&Z 세대
#### (2) Suggestion : Robo Advisor의 Event Driven Investment with Sentiment Analysis Report
#### (3) POC(Proof of Concept) : Nintendo
## 결론
### 외부데이터 및 참고자료


# 서론
## 1. 2020년 급격히 증가하는 주식거래

In [270]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import plotly.io as pio
pd.options.plotting.backend = "plotly"
pio.renderers.default = "notebook_connected"
#"국내/해외 투자 일자별 거래건수"
trd_kr=pd.read_csv(r"./data/2_trd_kr.csv")
trd_kr_info=trd_kr.copy()
trd_kr_info['주식시장']='국내'
trd_oss=pd.read_csv(r"./data/2_trd_oss.csv")
trd_oss_info=trd_oss.copy()
trd_oss_info['주식시장']='해외'
df=pd.concat([trd_kr_info,trd_oss_info],sort=False)
df['date'] =pd.to_datetime(df.orr_dt, format='%Y%m%d')
df=df.groupby(by=['주식시장','date']).count().reset_index()

fig = make_subplots(rows=2, cols=1,y_title='거래건수(1K=1000)')
fig.add_trace(  go.Scatter(x=df[df['주식시장']=='국내'].loc[:,'date'], y=df[df['주식시장']=='국내'].loc[:,'act_id'],name='국내투자'),
    row=1, col=1)
fig.add_trace(go.Scatter(x=df[df['주식시장']=='해외'].loc[:,'date'], y=df[df['주식시장']=='해외'].loc[:,'act_id'],name='해외투자'),
    row=2, col=1)
fig.update_layout(height=600, width=900, title={'text': "국내/해외투자 일자별 거래건 수",'x' : 0.5,},legend_title="투자구분",)
fig.update_xaxes(dtick="M1",tickformat="%b\n%Y",ticklabelmode="period")
fig.show()



국내와 해외투자 모두 2020년 초에 거래건수가 급격히 증가하고 있습니다. 특히 국내투자의 경우 3월을 기점으로 크게 증가하고 있습니다. 눈여겨 보아야할 점은 증가량은 국내투자가 2019년 연평균 약 5000여 건에서 2020년 20000여 건으로 해외투자보다 크게 증가했습니다. 하지만 해외투자의 경우 2019년 연평균 약 20여 건에서 200건으로 증가률은 더 높다는 점입니다. 이는 국내투자뿐만 아니라 해외투자에 대한 관심도 크게 증가했음을 보여줍니다.


### 주식투자에 대한 관심이 많이진 이유로는
### 첫 째, 계속되는 저금리 기조에 유동성이 주식시장으로 몰리고 있습니다. 



In [2]:
#금리추이
save_r=pd.read_csv(r"./data/한국은행_저축성수신금리.csv")
criteria_r=pd.read_csv(r"./data/한국은행_기준금리.csv")
save_r['type']='저축성수신금리'
criteria_r['type']='기준금리'
df=pd.concat([save_r,criteria_r],sort=False)
fig = px.line(df, x="date", y="r", color="type")
fig.update_layout(title={'text': "2019/2020 금리추이",'x' : 0.5,},xaxis_title="",yaxis_title="금리(%)",legend_title="금리구분")
fig.update_xaxes(dtick="M1",tickformat="%b\n%Y",ticklabelmode="period")
fig.show()

위 표는 한국은행에서 제공하는 기준금리와 저축성수신금리 자료입니다. 2019년부터 저금리 기조를 이어갔지만 특히 2020년 3월에는 기준금리를 1.25%에서 0.75%로 40% 감소했습니다.저축성수신금리도 이와 비슷하게 하락했는데 이러한 저금리 기조에서는 예금을 통한 이자소득이 너무 낮아 많은 유동성이 주식시장으로 이동했음을 짐작할 수 있습니다.<br>*데이터 출처: 한국은행

### 둘 째, COVID-19로 인한 주가 변동성 증가로 주식시장에서 큰 수익율을 올릴 수 있었습니다.  


In [3]:
#변동성 지수
kovix_index=pd.read_csv(r'./data/KOVIX.csv')
vix_index=pd.read_csv(r'./data/VIX.csv')

fig = make_subplots(rows=2, cols=1,y_title='INDEX')
fig.add_trace(go.Scatter(x=kovix_index['Date'], y=kovix_index['Close'],name='KOVIX_INDEX'),row=1, col=1)
fig.add_trace(go.Scatter(x=vix_index['Date'], y=vix_index['Close'],name='VIX_INDEX'),row=2, col=1)
fig.update_layout(height=600, width=950, title={'text': "2019/2020년 KOVIX/VIX INDEX",'x' : 0.5,},legend_title="지수구분")
fig.update_xaxes(dtick="M1",tickformat="%b\n%Y",ticklabelmode="period")
fig.show()


VIX INDEX란 시카고옵션거래소(CBOE)에서 거래되는 S&P500 지수옵션의 변동성을 나타내는 지표로 지수옵션에 대한 향후 30일간의 변동성에 대한 투자기대 지수를 나타냅니다. 예를 들면 VIX 30(%)이라고 하면 앞으로 한 달간 주가가 30%의 등락을 거듭할 것이라고 예상하는 투자자들이 많다는 것을 의미합니다. KOVIX INDEX란 한국형변동성지수로 코스피200의 옵션 가격을 이용해 옵션 투자자들이 예상하는 주식시장의 미래 변동성을 측정하는 지수입니다.<br>
위 표에 따르면 코로나 확산 이후 국내증시와 미국증시 모두 변동성이 급격하게 증가했다가 6월 이후로 안정적인 양상을 띄고 있습니다. 변동성이 높다는 것은 예년에 비해 높은 수익률을 올릴 가능성이 더 커짐을 의미합니다. 물론 아주 큰 손실을 입을 수 있는 가능성이 크다는 것을 의미하기도 합니다. 하지만 대부분의 투자자들은 변동성 국면에서 자신은 수익을 올릴 것이라 믿기 때문에 많은 투자자들이 일확천금을 노리고 주식시장에 참여했다고 볼 수 있습니다.<br>*데이터 출처: 한국거래소, Yahoo finance

### 셋 째, 주택가격이 상승해 근로소득만으로는 자가주택 마련이 어려워졌습니다. 

In [4]:
#집값 지수 추이
house_index=pd.read_csv(r"./data/house_index.csv",encoding='euc-kr')

fig = px.line(house_index, x="Date", y="index", color="type")

fig.update_layout(title={'text': "2019/2020 집값 지수 추이",'x' : 0.5,},xaxis_title="",yaxis_title="INDEX",legend_title="지역구분")
fig.update_xaxes(dtick="M1",tickformat="%b\n%Y",ticklabelmode="period")
fig.show()

집값 지수는 2017년 11월의 집값을 100으로 했을 때 집값을 나타냅니다. 2019년 5월까지 하락하던 집값은 서울을 시작으로 전국적으로 크게 상승하기 시작했습니다. 이러한 집값 상승이 이슈가 되자 Y&Z 세대는 근로소득만으로는 집을 구입하기 힘들다는 것을 깨닫고 재테크에 관심을 갖기 시작했습니다. 이러한 집값의 상승 또한 주식시장이 뜨겁게 타오는 원인이 되었습니다.
<br>*데이터 출처: 한국부동산원


## 2. 주식 열풍 속 Y&Z 세대
### 새로운 투자자 Y&Z세대

In [5]:
#고객정보, 계좌정보
cus_info=pd.read_csv(r"./data/2_cus_info.csv")
act_info=pd.read_csv(r"./data/2_act_info.csv")
cus_data=cus_info.copy()
#left join
cus_act_info=pd.merge(cus_info,act_info,how='left',on='cus_id')
#최초 신규 고객(New Resistered User) 여부
first_act_opn=cus_act_info.sort_values(['act_opn_ym']).groupby('cus_id').act_opn_ym.first().reset_index()
#2020년도 개설
first_act_opn['NRU_2020']=0
first_act_opn.loc[(first_act_opn.act_opn_ym>=202001)&(first_act_opn.act_opn_ym<=202012),'NRU_2020']=1
cus_data['NRU_2020']=cus_data.merge(first_act_opn,on='cus_id', how='left').iloc[:,-1]

cus_data.loc[(cus_data.cus_age<20),'cus_gen']='19세 이하'
cus_data.loc[(cus_data.cus_age>=20)&(cus_data.cus_age<30),'cus_gen']='20대'
cus_data.loc[(cus_data.cus_age>=30)&(cus_data.cus_age<40),'cus_gen']='30대'
cus_data.loc[(cus_data.cus_age>=40)&(cus_data.cus_age<50),'cus_gen']='40대'
cus_data.loc[(cus_data.cus_age>=50)&(cus_data.cus_age<60),'cus_gen']='50대'
cus_data.loc[(cus_data.cus_age>=60),'cus_gen']='60세 이상'
labels = [ "19세 이하","20대","30대","40대","50대","60세 이상"]
fig = go.Figure(data=[go.Pie(labels=labels, values=cus_data[cus_data.NRU_2020==1].groupby('cus_gen').count().reset_index()['cus_id'],
        sort=False)])
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(title={'text': "2020년 세대별 최초 신규 계좌 개설 비율",'x' : 0.45,},legend_title="세대구분")
fig.show()

2020년 최초 신규 계좌 개설(고객의 첫 계좌)에서 Y&Z세대의 비율은 51.7%로 절반 이상을 차지합니다. 이는 Y&Z세대가 새로운 구성원으로서 주식투자에 큰 관심을 보이고 있다고 해석할 수 있습니다.

### 증가하는 Y&Z세대의 주식투자

In [6]:
iem_info=pd.read_csv(r"./data/2_iem_info.csv")
#총거래액
trd_kr_info.loc[:,'total_pr']=trd_kr.loc[:,'cns_qty']*trd_kr.loc[:,'orr_pr']
trd_oss_info.loc[:,'total_pr']=trd_oss.loc[:,'cns_qty']*trd_oss.loc[:,'orr_pr']*trd_oss.loc[:,'trd_cur_xcg_rt']
#국내거래 해외거래 concat
trd=pd.concat([trd_kr_info,trd_oss_info],sort=False)
trd.loc[:,'iem_cd']=trd.loc[:,'iem_cd'].str.strip()
#주식정보와 merge
trd=pd.merge(trd,iem_info,how='left',on='iem_cd')
trd['date'] =pd.to_datetime(trd.orr_dt, format='%Y%m%d')
#고객정보,계좌정보,거래내역 merge/ 한번도 거래하지 않은 계좌는 drop
cus_act_trd_info=pd.merge(cus_act_info,trd,how='left',on='act_id')
cus_act_trd_info=cus_act_trd_info.dropna(subset=['cns_qty'])
cus_act_trd_info.reset_index(inplace=True)

df=cus_act_trd_info[(cus_act_trd_info.cus_age>=20)&(cus_act_trd_info.cus_age<40)].groupby(['주식시장','date']).sum().reset_index()

fig = make_subplots(rows=2, cols=1,y_title='총거래액(원)')
fig.add_trace(go.Scatter(x=df[df['주식시장']=='국내'].loc[:,'date'], y=df[df['주식시장']=='국내'].loc[:,'total_pr'],name='국내투자'),
    row=1, col=1)
fig.add_trace(go.Scatter(x=df[df['주식시장']=='해외'].loc[:,'date'], y=df[df['주식시장']=='해외'].loc[:,'total_pr'],name='해외투자'),
    row=2, col=1)
fig.update_layout(height=600, width=900, title={'text': "국내/해외투자 일자별 총거래액",'x' : 0.5,},legend_title="투자구분",)
fig.update_xaxes(dtick="M1",tickformat="%b\n%Y",ticklabelmode="period")
fig.show()



2020년 3월을 기점으로 Y&Z세대의 주식투자 총거래액이 크게 증가하고 있습니다. 이는 위에서 살펴보았던 한국은행의 기준금리 인하시기, VIX지수가 급등한 시기와 거의 일치합니다. 또한 COVID-19가 크게 창궐했던 시기와 일치합니다. 앞선 결과들과 이를 종합해보면 Y&Z세대는 COVID-19라는 위기 속에서 주식시장에 등장한 새로운 투자자로서 금융사의 새로운 고객입니다. 따라서 본론에서는 Y&Z세대의 특성을 분석하고 특화된 맞춤서비스를 제시하고자 합니다.

# 본론
## 1. Analysis
### (1) 데이터 전처리

- 데이터 불러오기  
- 종목정보 데이터(Iem_info) 전처리
- 거래내역 데이터(Trd_kr, Trd_oss) 전처리
- 고객정보 데이터(Cus_Info) 전처리
- 고객별 거래정보 데이터(Cus_act_trd_info) 생성 
- 고객-거래정보 Matrix 생성

### 데이터 불러오기
데이터 및 필요한 module을 불러오는 과정입니다.

In [311]:
import numpy as np
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import statistics
%matplotlib inline
from pykrx import stock

#font_path = '../fonts/NanumGothic.ttf'
font_name = matplotlib.font_manager.FontProperties(fname="C:/Windows/Fonts/HANBatang.ttf").get_name()
matplotlib.rc('font', family=font_name)
#matplotlib.rc('font', family="NanumBarunGothic")
#font_name = fm.FontProperties(fname=font_path, size=15).get_name()
matplotlib.use('Agg')

plt.rc('font', family=font_name, size=15)
plt.rcParams["figure.figsize"] = (16, 8)
register_matplotlib_converters()

#Load data
act_info=pd.read_csv(r"./data/2_act_info.csv")
cus_info=pd.read_csv(r"./data/2_cus_info.csv")
iem_info=pd.read_csv(r"./data/2_iem_info.csv")
trd_oss=pd.read_csv(r"./data/2_trd_oss.csv")
trd_kr=pd.read_csv(r"./data/2_trd_kr.csv")
#문자열 앞,뒤 공백 제거
iem_info_new=iem_info.copy()
for row in iem_info_new:
    iem_info_new[row]=iem_info_new[row].str.strip()


### 종목 정보 데이터(IEM_INFO) 전처리

- 문자열 양 끝의 공백 제거
- 종목별 정보 추가
    - 상장시장 기준 국가(Nation): <br>
        종목이 상장된 주식시장이 위치한 국가
            {'대한민국','미국','중국','기타'}
        
    - 시장(Market): <br>
        한국 시장에 상장된 주식/ELW/신주인수권의 경우:
            {'KOSPI(유가증권시장)','KOSDAQ'} 로 구분
        국내외 시장에 상장된 ETP(ETF 및 ETN 통칭)의 경우:
            {'대한민국','해외'}(투자 대상 자산군이 속한 국가)로 구분
            
    - 종류(Type): <br>
        한국 시장 상장종목의 경우:
            {'보통주', '우선주', 'ETP', 'ELW', '신주인수권'} 
        해외 시장 상장종목의 경우:
            {'일반주/기타', 'ETP}
       
    - 기업규모(Size):<br>
        {대형주, 중형주, 소형주} (KOSPI만 분류)
    <br><br>
    - ETP 종류(Type2):<br>
         국내외 시장의 ETP 종류별 구분
            {'주식형', '채권형','파생형'} 
        


### 국내 상장종목: 국가, 시장, 종류 구분
 - Python 패키지 pykrx 이용 - KOSPI, KOSDAQ 시장 상장종목 1차 분류
 - 종목코드 및 종목명에 대한 정규표현식 적용하여 2차 분류

In [312]:
#시장별 ticker 목록 및 개수
kospi_tickers = stock.get_market_ticker_list("20201216")
kospi_tickers_old = stock.get_market_ticker_list("20190102")
etf_tickers = stock.get_etf_ticker_list("20201216")
etf_tickers_old = stock.get_etf_ticker_list("20190102")
kosdaq_tickers = stock.get_market_ticker_list("20201216",market='KOSDAQ')
kosdaq_tickers_old = stock.get_market_ticker_list("20190102",market='KOSDAQ')
#국적,시장종류,주식타입 

# KOSPI 보통주
for code in kospi_tickers:
    code='A'+str(code)
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Nation']='대한민국'
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Market']='KOSPI'
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Type']='보통주'
for code in kospi_tickers_old:
    code='A'+str(code)
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Nation']='대한민국'
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Market']='KOSPI'
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Type']='보통주'
    
#KOSDAQ 보통주
for code in kosdaq_tickers:
    code='A'+str(code)
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Nation']='대한민국'
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Market']='KOSDAQ'
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Type']='보통주'
#KOSDAQ 보통주
for code in kosdaq_tickers_old:
    code='A'+str(code)
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Nation']='대한민국'
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Market']='KOSDAQ'
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Type']='보통주'
    
#KOSDAQ 우선주 3개 참고 A08537M(루트로닉3우)는 데이터에 없음
kosdaq_preference_shares=['A021045','A08537M','A032685']
for code in kosdaq_preference_shares:
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Nation']='대한민국'
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Market']='KOSDAQ'
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Type']='우선주'
    
#KOSPI 우선주 
iem_info_new.loc[iem_info_new['iem_krl_nm'].str.contains('.*우선주.*')&iem_info_new['Type'].isnull(),'Nation']='대한민국'
iem_info_new.loc[iem_info_new['iem_krl_nm'].str.contains('.*우선주.*')&iem_info_new['Type'].isnull(),'Market']='KOSPI'
iem_info_new.loc[iem_info_new['iem_krl_nm'].str.contains('.*우선주.*')&iem_info_new['Type'].isnull(),'Type']='우선주'


#ELW
iem_info_new.loc[iem_info_new['iem_eng_nm'].str.contains('.*ELW.*')&iem_info_new['iem_cd'].str.contains(r'^J.*'),'Nation']='대한민국'
iem_info_new.loc[iem_info_new['iem_eng_nm'].str.contains('.*ELW.*')&iem_info_new['iem_cd'].str.contains(r'^J.*'),'Market']='KOSPI'
iem_info_new.loc[iem_info_new['iem_eng_nm'].str.contains('.*ELW.*')&iem_info_new['iem_cd'].str.contains(r'^J.*'),'Type']='ELW'


#신주인수권
iem_info_new.loc[iem_info_new['iem_eng_nm'].str.contains('.*R$')&iem_info_new['iem_cd'].str.contains(r'^J.*'),'Nation']='대한민국'
iem_info_new.loc[iem_info_new['iem_eng_nm'].str.contains('.*R$')&iem_info_new['iem_cd'].str.contains(r'^J.*'),'Market']='KOSPI'
iem_info_new.loc[iem_info_new['iem_eng_nm'].str.contains('.*R$')&iem_info_new['iem_cd'].str.contains(r'^J.*'),'Type']='신주인수권'

#iem_info_new.loc[iem_info_new['iem_cd'].str.contains(r'^A.*|^Q.*|^J.*')&iem_info_new['Type'].isnull()] 로 확인


### 기업규모
- KOSPI에 상장된 기업을 한국거래소에서 제공하는 대형주, 중형주, 소형주 종목 정보 이용하여 분류(2020.12.26 기준)

In [313]:
#대형주 중형주 소형주 KRX기준
large_tickers = stock.get_index_portfolio_deposit_file("1002")
medium_tickers = stock.get_index_portfolio_deposit_file("1003")
small_tickers = stock.get_index_portfolio_deposit_file("1004")

#대형주
for code in large_tickers:
    code='A'+str(code)
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Size']='대형주'
#중형주
for code in medium_tickers:
    code='A'+str(code)
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Size']='중형주'
#소형주
for code in small_tickers:
    code='A'+str(code)
    iem_info_new.loc[iem_info_new['iem_cd']==code,'Size']='소형주'



### 국내 시장 ETP 
 - 종목명에 정규표현식을 적용하여 '주식형','채권형', '파생형'으로 구분<br>
 - 원자재형, 통화형, 합성형 등의 ETP는 모두 '파생형' ETP로 통칭

In [314]:
# 국내 시장 거래 종목 정보 
iem_info_new.loc[(iem_info_new['iem_eng_nm'].str.contains('.*ETF.*')|iem_info_new['iem_krl_nm'].str.contains('.*증권상장지수투자.*'))&\
                 iem_info_new['iem_cd'].str.contains(r'^A.*'),'Type']='ETP'
#텍스트 잘림으로 추가
iem_info_new.loc[iem_info_new['iem_cd']=='A341850','Type']='ETP'

#ETN
iem_info_new.loc[iem_info_new['iem_cd'].str.contains(r'^Q\d{6}'),'Type']='ETP'

#상장된 국가는 대한민국
iem_info_new.loc[ iem_info_new['Type']=='ETP','Nation']='대한민국'
#시장은 해외
iem_info_new.loc[(iem_info_new['iem_krl_nm'].str.contains\
                  ('.*S&P.*|.*중국.*|.*미국.*|.*글로벌.*|.*Japan.*|.* US .*|.*차이.*|.*베트남.*|.*일본.*|.*유로.*|.*신흥국.*|.*나스닥.*|.*라틴.*|.*싱가포르.*|.*선진국.*|.*인도.*|.*인디아.*|.*Chi.*|.*러시아.*|.*아시아.*|.*멕시코.*|.*MSCI.*|.*필리핀.*|.*호주.*'))&\
                 (iem_info_new['Type']=='ETP') ,'Market']='해외'
iem_info_new.loc[(iem_info_new['iem_krl_nm'].str.contains\
                  ('.*구리.*|.*은 선물.*|.*은선물.*|.*팔라듐.*|.*천연가스.*|.*원유.*|.*아연.*|.*콩.*|.*옥수수.*|.*골드선물.*|.*철광석.*|.*니켈.*|.*금 선물.*|.*금속.*|.*금은.*'))&\
                 (iem_info_new['Type']=='ETP') ,'Market']='해외'
#해외 이외는 국내
iem_info_new.loc[(iem_info_new['Type']=='ETP')&(iem_info_new['Market']!='해외'),'Market']='대한민국'

#주식형/파생형/채권형 구분
iem_info_new.loc[(iem_info_new['iem_krl_nm'].str.contains\
                  ('.*선물.*|.*인버스.*|.*레버리지.*|.*2X.*|.*양매도.*|.*파생.*|.*액티브.*|.*리츠.*|.*-파|.*-'))&\
                 (iem_info_new['Type']=='ETP') ,'Type2']='파생형'

iem_info_new.loc[(iem_info_new['iem_krl_nm'].str.contains\
                  ('.*\(채권\).*|.*\[채권\].*|.*\[채권'))&\
                 (iem_info_new['Type']=='ETP') ,'Type2']='채권형'

iem_info_new.loc[(iem_info_new['Type']=='ETP')&(iem_info_new['Type2'].isnull()) ,'Type2']='주식형'

### 해외 상장종목: 국가, 시장, 종류 구분
 - 종목코드 및 종목명에 대한 정규표현식 적용하여 1차 분류

In [315]:
#해외 주식 분류
#미국
iem_info_new.loc[iem_info_new['iem_cd'].str.contains(r'^US.*'),'Nation']='미국'
iem_info_new.loc[iem_info_new['iem_cd'].str.contains(r'^US.*'),'Market']='해외'
iem_info_new.loc[iem_info_new['iem_cd'].str.contains(r'^US.*'),'Type']='일반주/기타'

#중국
iem_info_new.loc[iem_info_new['iem_cd'].str.contains(r'^CNE.*'),'Nation']='중국'
iem_info_new.loc[iem_info_new['iem_cd'].str.contains(r'^CNE.*'),'Market']='해외'
iem_info_new.loc[iem_info_new['iem_cd'].str.contains(r'^CNE.*'),'Type']='일반주/기타'
iem_info_new.loc[iem_info_new['iem_eng_nm'].str.contains('.*ETF.*')&iem_info_new['iem_cd'].str.contains(r'^CNE.*'),'Nation']='중국'
iem_info_new.loc[iem_info_new['iem_eng_nm'].str.contains('.*ETF.*')&iem_info_new['iem_cd'].str.contains(r'^CNE.*'),'Market']='해외'
iem_info_new.loc[iem_info_new['iem_eng_nm'].str.contains('.*ETF.*')&iem_info_new['iem_cd'].str.contains(r'^CNE.*'),'Type']='ETP'
iem_info_new.loc[iem_info_new['iem_eng_nm'].str.contains('.*ETN.*')&iem_info_new['iem_cd'].str.contains(r'^CNE.*'),'Nation']='중국'
iem_info_new.loc[iem_info_new['iem_eng_nm'].str.contains('.*ETN.*')&iem_info_new['iem_cd'].str.contains(r'^CNE.*'),'Market']='해외'
iem_info_new.loc[iem_info_new['iem_eng_nm'].str.contains('.*ETN.*')&iem_info_new['iem_cd'].str.contains(r'^CNE.*'),'Type']='ETP'

#기타
iem_info_new.loc[iem_info_new['Nation'].isnull(),'Nation']='기타'
iem_info_new.loc[iem_info_new['Nation']=='기타','Market']='해외'
iem_info_new.loc[iem_info_new['Nation']=='기타','Type']='일반주/기타'
iem_info_new.loc[(iem_info_new['Nation']=='기타')&iem_info_new['iem_eng_nm'].str.contains('.*ETF.*'),'Nation']='기타'
iem_info_new.loc[(iem_info_new['Nation']=='기타')&iem_info_new['iem_eng_nm'].str.contains('.*ETF.*'),'Market']='해외'
iem_info_new.loc[(iem_info_new['Nation']=='기타')&iem_info_new['iem_eng_nm'].str.contains('.*ETF.*'),'Type']='ETP'
iem_info_new.loc[(iem_info_new['Nation']=='기타')&iem_info_new['iem_eng_nm'].str.contains('.*ETN.*'),'Nation']='기타'
iem_info_new.loc[(iem_info_new['Nation']=='기타')&iem_info_new['iem_eng_nm'].str.contains('.*ETN.*'),'Market']='해외'
iem_info_new.loc[(iem_info_new['Nation']=='기타')&iem_info_new['iem_eng_nm'].str.contains('.*ETN.*'),'Type']='ETP'


### 미국 시장 ETP 
 - 해외 시장 거래액 중 ??%를 차지하는 미국시장 상장종목 정보 추가
 - 기존 데이터에서 제공하는 종목 ISIN 코드 및 종목명을 이용한 정보 수집의 어려움 존재
     --> Bloomberg의 OpenFigi API를 이용하여 각 종목별 Ticker, 종류 등의 정보를 수집
 - ETP의 경우 Barchart.com에서 제공하는 ETP 세부 정보를 이용하여 '주식형','채권형', '파생형' 으로 구분

In [316]:
# 미국 시장 거래 종목 정보 
# 2020.12.26 기준
us=iem_info_new[iem_info_new.Nation=='미국'].iem_cd.values
jobs=[]
for i in range(len(us)):
    temp={'idType': 'ID_ISIN',  'exchCode': 'US'}
    temp['idValue']=us[i]
    jobs.append(temp)
# Openfigi 이용하여 종목별 정보 수집
def map_jobs(jobs):
    if openfigi_apikey:
        openfigi_headers['X-OPENFIGI-APIKEY'] = openfigi_apikey
    response = requests.post(url=openfigi_url, headers=openfigi_headers,
                             json=jobs)
    if response.status_code != 200:
        raise Exception('Bad response code {}'.format(str(response.status_code)))
    return response.json()
########################키 삭제#########################
openfigi_apikey = 'e557386f-cdf5-4617-88b6-f3c38a0a9eb7'
openfigi_url = 'https://api.openfigi.com/v2/mapping'
openfigi_headers = {'Content-Type': 'text/json'}
########################################################
job_results=[]
for i in range(12):
    job_results_temp=map_jobs(jobs[i*100:i*100+100])
    job_results=job_results+job_results_temp


# 종목별 ticker 및 종류 데이터프레임화 
def get_ticker_info(i,x):
    d=[]
    try:
        d.append(us[i])
        d.append(x['data'][0]['ticker'])
        d.append(x['data'][0]['marketSector'])
        d.append(x['data'][0]['securityType'])
        d.append(x['data'][0]['securityType2'])
        return d
    except:
        return ''
job_res=[get_ticker_info(i,x) for i,x in enumerate(job_results)]
job_res=[x for x in job_res if x]
df = pd.DataFrame (job_res,columns=['ISIN','ticker','marketSector','securityType','securityType2'])


In [317]:
# 종목별 Type 추출
df.loc[:,'Type']='일반주/기타'
df.loc[df.securityType=='ETP','Type']='ETP'
# 미국 종목 정보에 병합 
us_iem=iem_info_new[iem_info_new.Nation=='미국']
us_iem_info=us_iem.loc[:,['iem_cd', 'iem_eng_nm']].merge(df.loc[:,['ISIN','ticker', 'Type']],right_on='ISIN', left_on='iem_cd', how='left')
#us_iem_info.to_csv('us_iem_final.csv', index=False, encoding='euc-kr')

#OpenFigi에서 누락된 ETP 항목 정보 추가
us_iem_info=pd.read_csv('./data/us_iem_final2.csv', encoding='euc-kr')
us_iem_info['Type'].fillna('일반주/기타', inplace=True)
us_iem_info.drop('ISIN', axis=1, inplace=True)
us_iem_info['Market']='해외'
us_iem_info['Nation']='미국'


In [318]:
#ETP 리스트
etf_fi=pd.read_csv(r"./data/etfs/etf_fixed_income.csv")
etf_alt=pd.read_csv(r"./data/etfs/etf_alternative.csv")
etf_com=pd.read_csv(r"./data/etfs/etf_commodity.csv")
etf_cur=pd.read_csv(r"./data/etfs/etf_currency.csv")
etf_eq2=pd.read_csv(r"./data/etfs/etf_equity_lev1_1.csv")
etf_eq1=pd.read_csv(r"./data/etfs/etf_equity_lev.csv")
etf_eq3=pd.read_csv(r"./data/etfs/etf_equity_lev1_2.csv")

etf_mix=pd.read_csv(r"./data/etfs/etf_mixed.csv")
j=0
types=['채권형', '파생형', '파생형','파생형','주식형','파생형','주식형', '파생형']
for i in [etf_fi, etf_alt, etf_com, etf_cur, etf_eq2, etf_eq1, etf_eq3, etf_mix]:
    i['etf_type']=types[j]
    j+=1
etfs=pd.concat([etf_fi, etf_alt, etf_com, etf_cur, etf_eq2, etf_eq1, etf_eq3, etf_mix]).loc[:,['Symbol','Name','Leverage', 'etf_type']]

In [319]:
#ETP의 분류
etf_iem_info=us_iem_info[us_iem_info.Type=='ETP']
etf_iem_info=etf_iem_info.merge(etfs, left_on='ticker', right_on='Symbol' , how='left')
etf_iem_info.etf_type.fillna("파생형", inplace = True)  # ETP 리스트에 누락된 ETP 모두 레버리지형 ETF에 해당하여 파생형으로 분류

#ETP 정보 병합 
us_info_etf=us_iem_info.merge(etf_iem_info.loc[:,
                                ['iem_cd','ticker','etf_type']], on='iem_cd', how='left')
us_info_etf.drop_duplicates(inplace=True)

us_info_etf.rename(columns={'etf_type':'Type2',
                          'ticker_y':'Ticker'},
                 inplace=True)

#미국 시장 주식 정보 통합 
us_new=iem_info_new.merge(us_info_etf, how='left', on='iem_cd').loc[:,['Type_y', 'Type2_y']]
us_new.rename(columns = {'Type2_y':'Type2', 'Type_y':'Type'},inplace=True)

iem_info_new.update(us_new)
#iem_info_new.to_csv('iem_info_final.csv', index=False, encoding='euc-kr')


### 거래내역 데이터(Trd_kr, Trd_oss) 전처리

- 신규 변수 'total_pr' 생성:
    - 체결수량(CNS_QTY) 및 체결가격(ORR_PR)을 곱해 '거래대금'(이하 '거래액') 열 추가
    - 외화 거래의 경우 거래통화 환율(TRD_CUR_XCG_RT)을 함께 곱하여 계산
    - 기존 체결수량 변수의 경우 해당 종목의 주당가격에 크게 의존하여 비교가 어렵다는 문제점 보완 목적
 
 
- 신규 변수 'date', 'year', 'month', 'weekday' 생성:
    - '일별', '월별', '연도별' 등의 비교 목적
    
    
- 종목 정보 데이터와 병합하여 종목별 거래내역 데이터 생성

In [320]:
#총거래액
trd_kr.loc[:,'total_pr']=trd_kr.loc[:,'cns_qty']*trd_kr.loc[:,'orr_pr']
trd_oss.loc[:,'total_pr']=trd_oss.loc[:,'cns_qty']*trd_oss.loc[:,'orr_pr']*trd_oss.loc[:,'trd_cur_xcg_rt']
#국내거래 해외거래 concat
trd=pd.concat([trd_kr,trd_oss],sort=False)
trd.loc[:,'iem_cd']=trd.loc[:,'iem_cd'].str.strip()
trd.replace({'sby_dit_cd': {1: 'sell', 2: 'buy'},
            'orr_mdi_dit_cd': {1: '영업점', 2: '유선', 3:'MTS', 4: 'HTS'}}, inplace=True) # 매매구분 및 주문매체 값 변경

#날짜 정보 추가
trd['date'] =pd.to_datetime(trd.orr_dt, format='%Y%m%d')
trd['year']=trd['date'].dt.year
trd['month']=trd['date'].dt.month
trd['weekday']=trd['date'].dt.weekday 

#주식정보와 merge
trd=pd.merge(trd,iem_info_new,how='left',on='iem_cd')

### 고객정보 데이터(Cus_Info) 전처리

- 고객정보 중 'gen'(세대) 변수 추가

- 'Y&Z'(20대 또는 30대), 'Young'(20세 미만), 'Senior'(40세 이상)로 고객 구분
    
- 계좌정보 데이터와 병합하여 고객별 계좌 데이터 생성

In [321]:
#고객정보와 계좌정보 merge

cus_info.loc[cus_info['cus_age']==0,'gen']='Young'
cus_info.loc[(cus_info['cus_age']>0)&(cus_info['cus_age']<40),'gen']='Y&Z'
cus_info.loc[cus_info['gen'].isnull(),'gen']='Senior'
cus_act_info=pd.merge(cus_info,act_info,how='left',on='cus_id')


### 고객별 거래내역 데이터 (cus_act_trd_info) 
    
- 고객별 계좌 데이터와 그에 상응하는 거래 데이터를 결합하여 고객별 거래내역 데이터생성
- 주어진 기간 내 한 건의 거래내역도 존재하지 않는 계좌는 제외

In [322]:
#고객정보,계좌정보,거래내역 merge/ 한번도 거래하지 않은 계좌는 drop
cus_act_trd_info=pd.merge(cus_act_info,trd,how='left',on='act_id')
cus_act_trd_info=cus_act_trd_info.dropna(subset=['cns_qty'])
cus_act_trd_info.reset_index(inplace=True)

### 고객-거래정보 Matrix 생성
- 고객별 거래내역 데이터를 가공하여 고객별 거래정보 

In [323]:
# 고객 정보 요약 matrix
cus_data=cus_info.copy()
#평균 총 거래액
temp=cus_act_trd_info.groupby('cus_id')['total_pr'].mean().reset_index()
cus_data['amt']=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1]

In [324]:
#계좌 개수
cus_data['num_act']=cus_data.merge(cus_act_trd_info.groupby('cus_id').act_id.nunique(),on='cus_id', how='left').iloc[:,-1]

#2020 신규 계좌 수
temp=cus_act_info[cus_act_info.act_opn_ym>=202001].groupby('cus_id').act_id.nunique()
cus_data['num_act_2020']=cus_data.merge(temp,on='cus_id', how='left').iloc[:,-1]

#2019 신규 계좌 수
temp=cus_act_info[cus_act_info.act_opn_ym<202001].groupby('cus_id').act_id.nunique()
cus_data['num_act_2019']=cus_data.merge(temp,on='cus_id', how='left').iloc[:,-1]


#신규고객(New Resistered User) 여부

#첫계좌 개설일 기준 2020.04 이후 개설 고객 대상
first_act_opn=cus_act_info.sort_values(['act_opn_ym']).groupby('cus_id').act_opn_ym.first().reset_index()
first_act_opn['NRU']=0
first_act_opn.loc[first_act_opn.act_opn_ym>202003,'NRU']=1
cus_data['NRU']=cus_data.merge(first_act_opn,on='cus_id', how='left').iloc[:,-1]
#2020.01 이후 개설
first_act_opn['NRU_2020']=0
first_act_opn.loc[first_act_opn.act_opn_ym>201912,'NRU_2020']=1
cus_data['NRU_2020']=cus_data.merge(first_act_opn,on='cus_id', how='left').iloc[:,-1]

# 총 거래건수
temp=cus_act_trd_info.groupby(['cus_id']).total_pr.count().reset_index()
cus_data['qty']=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1]


In [325]:
# Portfolio 구성 종목 수 
#전체 종목 수
temp=cus_act_trd_info.groupby(['cus_id']).iem_cd.nunique().reset_index()
cus_data['n_comp']=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1]

#국내시장 종목 수 (상장시장 기준)
temp=cus_act_trd_info[cus_act_trd_info.Nation=='대한민국'].groupby('cus_id').iem_cd.nunique().reset_index()
cus_data['n_comp_domestic']=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1]

#해외 종목 수 
temp=cus_act_trd_info[cus_act_trd_info.Nation!='대한민국'].groupby('cus_id').iem_cd.nunique().reset_index()
cus_data['n_comp_global']=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1]


In [326]:
# 하루 평균 거래건수 
temp=cus_act_trd_info.groupby(['cus_id','date'])['total_pr'].count().reset_index()\
                                              .groupby(['cus_id']).total_pr.mean().reset_index()
cus_data['qty_daily']=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1]

#2020년 하루 평균 거래건수 
temp=cus_act_trd_info[cus_act_trd_info.year==2020].groupby(['cus_id','date'])['total_pr'].count().reset_index()\
                                              .groupby(['cus_id']).total_pr.mean().reset_index()
cus_data['qty_daily_2020']=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1]
#2019년 하루 평균 거래건수 
temp=cus_act_trd_info[cus_act_trd_info.year==2019].groupby(['cus_id','date'])['total_pr'].count().reset_index()\
                                              .groupby(['cus_id']).total_pr.mean().reset_index()
cus_data['qty_daily_2019']=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1]




In [327]:
# 시장별/국가별 거래액
for k in ['Nation','Market','Type','Size', 'orr_mdi_dit_cd','Type2']:
    for i in pd.unique(cus_act_trd_info[k]):
        temp=cus_act_trd_info[cus_act_trd_info[k]==i].groupby('cus_id')['total_pr'].mean().reset_index()
        cus_data['amt_'+str(k)+'_'+str(i)]=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1]
for k in ['orr_rtn_hur','month','weekday','year']:
    for i in pd.unique(cus_act_trd_info[k]):
        temp=cus_act_trd_info[cus_act_trd_info[k]==i].groupby('cus_id')['total_pr'].mean().reset_index()
        cus_data['amt_'+str(k)+'_'+str(int(i))]=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1]

# 총 시장별/국가별 거래건수

for k in ['Nation','Market','Type','Size', 'orr_mdi_dit_cd', 'Type2']:
    for i in pd.unique(cus_act_trd_info[k]):
        temp=cus_act_trd_info[cus_act_trd_info[k]==i].groupby('cus_id')['total_pr'].count().reset_index()
        cus_data['qty_'+str(k)+'_'+str(i)]=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1]
for k in ['orr_rtn_hur','weekday', 'year']:
    for i in pd.unique(cus_act_trd_info[k]):
        temp=cus_act_trd_info[cus_act_trd_info[k]==i].groupby('cus_id')['total_pr'].count().reset_index()
        cus_data['qty_'+str(k)+'_'+str(int(i))]=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1]



In [328]:

# 총 매수/매도/순매수액 
for i in pd.unique(cus_act_trd_info.sby_dit_cd):
    temp=cus_act_trd_info[cus_act_trd_info.sby_dit_cd==i].groupby('cus_id')['total_pr'].mean().reset_index()
    cus_data['amt_'+i]=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1].fillna(0) 
    
cus_data['amt_net']=cus_data['amt_buy']-cus_data['amt_sell']

# 총 시장별/국가별/크기별...  매수/매도/순매수액  

for k in ['Nation','Market','Type','Size', 'orr_mdi_dit_cd','Type2']:
    for i in pd.unique(cus_act_trd_info[k]):
        for j in pd.unique(cus_act_trd_info.sby_dit_cd):

            temp1=cus_act_trd_info[cus_act_trd_info[k]==i]
            temp=temp1[temp1.sby_dit_cd==j].groupby('cus_id')['total_pr'].mean().reset_index()
            cus_data['amt_'+str(k)+'_'+str(i)+'_'+str(j)]=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1].fillna(0) 
        cus_data['amt_'+str(k)+'_'+str(i)+'_net']=cus_data['amt_'+str(k)+'_'+str(i)+'_buy']-cus_data['amt_'+str(k)+'_'+str(i)+'_sell']

for k in ['orr_rtn_hur','weekday']:
    for i in pd.unique(cus_act_trd_info[k]):
        for j in pd.unique(cus_act_trd_info.sby_dit_cd):

            temp1=cus_act_trd_info[cus_act_trd_info[k]==i]
            temp=temp1[temp1.sby_dit_cd==j].groupby('cus_id')['total_pr'].mean().reset_index()
            cus_data['amt_'+str(k)+'_'+str(int(i))+'_'+str(j)]=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1].fillna(0) 
        cus_data['amt_'+str(k)+'_'+str(int(i))+'_net']=\
        cus_data['amt_'+str(k)+'_'+str(int(i))+'_buy']-cus_data['amt_'+str(k)+'_'+str(int(i))+'_sell']
        

In [329]:
# 총 매수/매도/순매수 거래건수
for i in pd.unique(cus_act_trd_info.sby_dit_cd):
    temp=cus_act_trd_info[cus_act_trd_info.sby_dit_cd==i].groupby('cus_id')['total_pr'].count().reset_index()
    cus_data['qty_'+i]=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1].fillna(0) 
    
cus_data['qty_net']=cus_data['amt_buy']-cus_data['amt_sell']

# 총 시장별/국가별/크기별...  매수/매도/순매수 거래건수  

for k in ['Nation','Market','Type','Size', 'orr_mdi_dit_cd', 'Type2']:
    for i in pd.unique(cus_act_trd_info[k]):
        for j in pd.unique(cus_act_trd_info.sby_dit_cd):

            temp1=cus_act_trd_info[cus_act_trd_info[k]==i]
            temp=temp1[temp1.sby_dit_cd==j].groupby('cus_id')['total_pr'].count().reset_index()
            cus_data['qty_'+str(k)+'_'+str(i)+'_'+str(j)]=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1].fillna(0) 
        cus_data['qty_'+str(k)+'_'+str(i)+'_net']=cus_data['qty_'+str(k)+'_'+str(i)+'_buy']-cus_data['qty_'+str(k)+'_'+str(i)+'_sell']

for k in ['orr_rtn_hur','weekday']:
    for i in pd.unique(cus_act_trd_info[k]):
        for j in pd.unique(cus_act_trd_info.sby_dit_cd):

            temp1=cus_act_trd_info[cus_act_trd_info[k]==i]
            temp=temp1[temp1.sby_dit_cd==j].groupby('cus_id')['total_pr'].count().reset_index()
            cus_data['qty_'+str(k)+'_'+str(int(i))+'_'+str(j)]=cus_data.merge(temp, on='cus_id', how='left').iloc[:,-1].fillna(0) 
        cus_data['qty_'+str(k)+'_'+str(int(i))+'_net']=\
            cus_data['qty_'+str(k)+'_'+str(int(i))+'_buy']-cus_data['qty_'+str(k)+'_'+str(int(i))+'_sell']



### (2) Y&Z 세대의 주식투자 특징
전처리를 통해 형성된 고객별 거래정보 데이터를 바탕으로 Y&Z 세대의 특징을 시각화하여 분석하고자 합니다.

### 해외투자에 관심이 많은 Y&Z세대 

In [345]:
df1=cus_data.groupby('gen').count().reset_index()
df2=cus_data[cus_data.n_comp_global.notna()].groupby('gen').count().reset_index()

fig = make_subplots(rows=1, cols=2,subplot_titles=['전체 고객 분포','해외투자 유경험 고객 분포'],horizontal_spacing = 0.15)
fig.add_trace(go.Bar(x=df1.gen, y=df1.cus_id,name='전체 고객 분포'),1, 1)
fig.add_trace(go.Bar(x=df2.gen, y=df2.cus_id,name='해외투자 유경험 고객 분포'),1, 2)
fig.update_layout(height=500, width=800, title={'text': "주식투자자 세대별 인구 분포 ",'x' : 0.5,}, showlegend=False)
fig['layout']['yaxis']['title']='명'
fig['layout']['yaxis2']['title']='명'
fig.update_xaxes(type='category')
fig.show()

본격적인 분석에 앞서 주어진 샘플링된 1만 명의 고객정보를 통해 세대별 인구 분포를 파악합니다. 좌측의 그래프에 따르면 Senior(40대 이상)가 63%를 차지하고 있으며 Y&Z세대(20~30대)가 34%를 차지하고 있습니다. 하지만 우측의 그래프에 따르면 해외투자를 한 번이라도 경험해 본 투자자는 총 790명 중 Y&Z세대가 약 67%를 차지하고 있습니다. 이를 통해 Y&Z세대는 Senior에 비해 해외투자에 관심이 상대적으로 많다고 해석할 수 있습니다. 

### Y&Z세대 해외 직접 투자의 증가

In [349]:
df=cus_act_trd_info[cus_act_trd_info.gen=='Y&Z']
temp1=df[df.Nation!='대한민국'].groupby('year').sum().loc[:,['total_pr']].reset_index()
temp2=(df[df.Nation!='대한민국'].groupby('year').sum().loc[:,['total_pr']]/df.groupby('year').sum().loc[:,['total_pr']]).reset_index()
temp3=df[df.Nation!='대한민국'].groupby('year').count().loc[:,['index']].reset_index()
temp4=(df[df.Nation!='대한민국'].groupby('year').count().loc[:,['index']]/df.groupby('year').count().loc[:,['index']]).reset_index()
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=['해외투자 총 거래액','총 거래액 대비 해외투자 거래액 비율',
                                    '해외투자 총 거래건','총 거래건 대비 해외투자 거래 비율'],horizontal_spacing = 0.15)
fig.add_trace(go.Bar(x=temp1['year'].astype('int'), y=temp1['total_pr'],name='해외투자 총 거래액'),1, 1)
fig.add_trace(go.Bar(x=temp2['year'].astype('int'), y=temp2['total_pr']*100,name='총 거래액 대비 해외투자 거래액 비율'),1, 2)
fig.add_trace(go.Bar(x=temp3['year'].astype('int'), y=temp3['index'],name='해외투자 총 거래건'),2, 1)
fig.add_trace(go.Bar(x=temp4['year'].astype('int'), y=temp4['index']*100,name='해외투자 거래 비율'),2, 2)
fig.update_layout(height=700, width=800, title={'text': "2019/2020 Y&Z 세대 해외 직접 투자 비교",'x' : 0.5,}, showlegend=False)
fig['layout']['yaxis']['title']='원(1B=10억)'
fig['layout']['yaxis2']['title']='%'
fig['layout']['yaxis3']['title']='건(1K=1000)'
fig['layout']['yaxis4']['title']='%'
fig.update_xaxes(type='category')
fig.show()

해외 직접 투자는 외국 증권시장에 상장된 주식을 외화로 직접 투자하는 것입니다. 예를 들어 NH투자증권에서 테슬라 주식을 구입한다면 이는 해외 직접 투자입니다. 2019년에 비해 2020년에 Y&Z세대의 해외 직접 투자는 거래액과 거래건 모두 5배 이상 크게 증가했습니다. 또한 총 거래액 대비 해외투자 거래액의 비율과 총 거래건 대비 해외투자 거래의 비율 모두 3배이상 크게 증가했습니다. 이를 통해 2020년도에 Y&Z세대는 해외투자를 크게 늘렸음을 알 수 있습니다.   

### Y&Z세대 해외 간접 투자의 증가

In [350]:
df=cus_act_trd_info[cus_act_trd_info.gen=='Y&Z']
temp1=df[(df.Nation=='대한민국')&(df.Market=='해외')].groupby('year').sum().loc[:,['total_pr']].reset_index()
temp2=(df[(df.Nation=='대한민국')&(df.Market=='해외')].groupby('year').sum().loc[:,['total_pr']]/df.groupby('year').sum().loc[:,['total_pr']]).reset_index()
temp3=df[(df.Nation=='대한민국')&(df.Market=='해외')].groupby('year').count().loc[:,['index']].reset_index()
temp4=(df[(df.Nation=='대한민국')&(df.Market=='해외')].groupby('year').count().loc[:,['index']]/df.groupby('year').count().loc[:,['index']]).reset_index()
fig = make_subplots(rows=2, cols=2, 
                    subplot_titles=['해외투자 총 거래액','총 거래액 대비 해외투자 거래액 비율',
                                    '해외투자 총 거래건','총 거래건 대비 해외투자 거래 비율'],horizontal_spacing = 0.15)
fig.add_trace(go.Bar(x=temp1['year'].astype('int'), y=temp1['total_pr'],name='해외투자 총 거래액'),1, 1)
fig.add_trace(go.Bar(x=temp2['year'].astype('int'), y=temp2['total_pr']*100,name='총 거래액 대비 해외투자 거래액 비율'),1, 2)
fig.add_trace(go.Bar(x=temp3['year'].astype('int'), y=temp3['index'],name='해외투자 총 거래건'),2, 1)
fig.add_trace(go.Bar(x=temp4['year'].astype('int'), y=temp4['index']*100,name='해외투자 거래 비율'),2, 2)
fig.update_layout(height=700, width=800, title={'text': "2019/2020 Y&Z 세대 해외 간접 투자 비교",'x' : 0.5,}, showlegend=False)
fig['layout']['yaxis']['title']='원(1B=10억)'
fig['layout']['yaxis2']['title']='%'
fig['layout']['yaxis3']['title']='건(1K=1000)'
fig['layout']['yaxis4']['title']='%'
fig.update_xaxes(type='category')
fig.show()

해외 간접 투자는 국내 증권시장에 상장된 해외자산으로 구성된 펀드를 원화로 투자하는 것입니다. 예를 들어 NH투자증권에서 "TIGER 미국S&P 500 선물(H)를 구입하는 것이 해외 간접 투자입니다. 2019년에 비해 2020년에 Y&Z세대의 해외 간접 투자는 거래액과 거래건 모두 8배 이상 크게 증가하여 해외 직접 투자보다 큰 증가율을 보입니다. 또한 총 거래액 대비 해외투자 거래액의 비율과 총 거래건 대비 해외투자 거래의 비율 모두 3배이상 크게 증가했습니다. 이를 통해 2020년도에 Y&Z세대는 해외 직접 투자뿐만 아니라 해외 간접 투자 또한 늘렸음을 알 수 있습니다. 이러한 2020년도 Y&Z세대의 해외 주식 투자에 대한 큰 관심을 바탕으로 앞으로 해외투자를 중심으로 Y&Z세대의 특징을 분석해보도록 하겠습니다.

### Y&Z세대의 투자성향

In [333]:
temp=cus_data[cus_data.gen=='Y&Z']
df1=temp.groupby('ivs_icn_cd').count().reset_index()
temp=cus_data[(cus_data.gen=='Y&Z')&(cus_data.n_comp_global.notna())]
df2=temp.groupby('ivs_icn_cd').count().reset_index()

fig = make_subplots(rows=1, cols=2,subplot_titles=['전체 Y&Z세대','해외투자 유경험 Y&Z세대'],horizontal_spacing = 0.15)
fig.add_trace(go.Bar(x=df1.ivs_icn_cd, y=df1.cus_id,name='전체 Y&Z세대'),1, 1)
fig.add_trace(go.Bar(x=df2.ivs_icn_cd, y=df2.cus_id,name='해외투자 유경험 Y&Z세대'),1, 2)
fig.update_layout(height=500, width=800, title={'text': "Y&Z세대 고객투자성향 분포 ",'x' : 0.5,}, showlegend=False)
fig['layout']['yaxis']['title']='명'
fig['layout']['yaxis2']['title']='명'
fig.update_xaxes(type='category')
fig.show()

Y&Z세대는 위험중립형 투자자가 가장 많습니다. 적극투자형과 안정추구형이 그 뒤를 따릅니다. 하지만 전체 3417명의 Y&Z세대의 고객투자성향을 분석함에 있어서 해당사항 없음의 비율이 36%가량을 차지하기 때문에 정확한 분석이 어렵습니다. 이에 반해 해외투자 유경험자의 경우 그 분포를 앞선 전체 Y&Z세대와 비슷하지만 해당사항 없음의 비율이 낮아 비교적 정확한 분포라고 할 수 있습니다. 따라서 Y&Z세대의 해외투자 유경험자의 경우 위험중립형, 적극투자형, 안저추구형 순서로 많은 비율을 차지하고 있습니다.  

### 공격적인 Y&Z세대의 해외투자 

In [351]:
temp=cus_act_trd_info[(cus_act_trd_info.gen=='Y&Z')&(cus_act_trd_info.Nation!='대한민국')]
df1=temp.groupby('ivs_icn_cd').sum().loc[:,['total_pr']].reset_index()
df2=temp.groupby('ivs_icn_cd').count().loc[:,['index']].reset_index()

labels = ["해당사항 없음","정보제공 미동의","안정형","안정추구형","위험중립형","적극투자형","공격투자형"]
fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],subplot_titles=['총거래액(원)', '총거래건수(건)'])
fig.add_trace(go.Pie(labels=labels, values=df1.total_pr,name="총거래액"), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=df2.loc[:,'index'],name="총거래건수"), 1, 2)

fig.update_layout(title={'text': "Y&Z 세대의 해외투자에서 투자성향별 비율",'x' : 0.5,})
fig.show()


Y&Z세대의 투자성향 분포와 달리 실제로 해외투자에 적극적인 투자성향은 적극투자형과 공격투자형입니다. 총 거래액의 65%를 총 거래건수의 49.6%를 두 성향이 점유하고 있습니다. 이를 통해 Y&Z세대에서 실질적으로 해외투자를 이끌고 있는 것은 적극투자형과 공격투자형임을 알 수 있습니다.

### Y&Z세대가 많이 구입한 해외주식 종목 

In [352]:
temp=cus_act_trd_info[(cus_act_trd_info.gen=='Y&Z')&(cus_act_trd_info.Nation!='대한민국')]
df1=temp.groupby('iem_krl_nm').sum().loc[:,['total_pr']].sort_values(by=['total_pr'],ascending=False).head(10).reset_index()
df2=temp[(temp.ivs_icn_cd=='04')|(temp.ivs_icn_cd=='05')]
df2=df2.groupby('iem_krl_nm').sum().loc[:,['total_pr']].sort_values(by=['total_pr'],ascending=False).head(10).reset_index()

fig = make_subplots(rows=2, cols=1,subplot_titles=['모든 Y&Z 세대','적극/공격 투자형'] ,vertical_spacing = 0.15)
fig.add_trace(go.Bar(y=df1.iem_krl_nm, x=df1.total_pr,name='모든 Y&Z 세대', orientation='h'),1, 1)
fig.add_trace(go.Bar(y=df2.iem_krl_nm, x=df2.total_pr,name='적극/공격 투자형', orientation='h'),2, 1)
fig.update_layout(height=1000, width=950, title={'text': "Y&Z세대 해외주식 종목 TOP10",'x' : 0.5,}, showlegend=False)
fig['layout']['xaxis']['title']='원(1B=10억)'
fig['layout']['xaxis2']['title']='원(1M=100만원)'
fig.show()

Y&Z세대가 거래한 해외 주식 종목 중 테슬라가 가장 큰 거래액을 차지했습니다. 무려 10억원 이상의 거래가 발생했네요. TOP10 목록 중 주목해야할 종목은 ProShares:UltP Sht QQQ과 MICRSECTRS US BIG OIL IDX 3X LVG ETN 그리고 프로셰어즈 울트라프로 QQQ입니다. ProShares:UltP Sht QQQ의 경우 나스닥지수가 떨어질 것에 배팅하는 상품인 반면 프로셰어즈 울트라프로 QQQ는 나스닥 지수가 오를 것에 배팅하는 상품입니다. MICRSECTRS US BIG OIL IDX 3X LVG ETN의 경우 미국의 10개의 오일/에너지 기업의 주가가 오를 것에 배팅하는 상품입니다. 이 종목들의 공통점은 파생형 상품이고 모두 레버리지를 3배 사용하고 있습니다. 레버리지를 3배 사용한다는 것은 기초 자산이 1만큼 변동할 때 해당 상품은 3만큼 변한다는 것을 의미하므로 세 가지 상품 모두 초고위험군의 상품입니다. TOP10 목록에 위험상품이 3개나 있다는 것은 그만큼 Y&Z세대가 공격적인 투자를 한다는 것을 의미합니다. <br>
아래의 그래프는 Y&Z세대 중 적극/공격 투자형 투자자의 거래액 TOP10 종목입니다. 1위는 ProShares:UltP Sht QQQ가 차지했습니다. 이 목록에는 초고위험 상품이 4개나 포함되어 있습니다.

### Y&Z세대의 빈약한 해외투자 포트폴리오 다양성

In [335]:
temp=cus_act_trd_info[(cus_act_trd_info['Nation']!='대한민국')].groupby(by='cus_id').nunique().loc[:,['iem_cd']]
df1=pd.merge(temp,cus_info.loc[:,['cus_id','gen']],how='left',on='cus_id').groupby(by='gen').mean().reset_index()
temp=cus_act_trd_info[(cus_act_trd_info['Nation']=='대한민국')].groupby(by='cus_id').nunique().loc[:,['iem_cd']]
df2=pd.merge(temp,cus_info.loc[:,['cus_id','gen']],how='left',on='cus_id').groupby(by='gen').mean().reset_index()

fig = make_subplots(rows=1, cols=2,subplot_titles=['해외 주식투자 평균 거래 종목 수','국내 주식투자 평균 거래 종목 수'],
                   horizontal_spacing = 0.15)
fig.add_trace(go.Bar(x=df1.gen[:2], y=df1.iem_cd[:2],name='해외 주식투자 평균 거래 종목 수'),1, 1)
fig.add_trace(go.Bar(x=df2.gen[:2], y=df2.iem_cd[:2],name='국내 주식투자 평균 거래 종목 수'),1, 2)
fig.update_layout(height=500, width=800, title={'text': "세대별 국내/해외 투자 평균 거래 종목 수 비교 ",'x' : 0.5,},showlegend=False)
fig['layout']['yaxis']['title']='개'
fig['layout']['yaxis2']['title']='개'
fig.show()

*Don't put all your eggs in one basket* <br>
분산 투자의 중요성은 이미 속담이 있을 정도로 많은 사람이 알고 있습니다. 하지만 해외투자에 있어서 만큼은 아닌 것 같습니다. 국내 주식투자 평균 거래 종목 수는 평균적으로 30개가 넘어가는데 해외 주식 투자의 경우 10개가 되지 않습니다. 이러한 현상은 Senior에 비해 Y&Z세대에서 뚜렷하게 나타납니다. 따라서 Y&Z세대는 조금더 다양한 해외주식을 구입할 필요가 있어보입니다.

### 전화로도 주식구매가 가능하다구요? 

In [353]:
df1=cus_act_trd_info[cus_act_trd_info.gen=='Y&Z'].groupby('orr_mdi_dit_cd').count().reset_index()
df2=cus_act_trd_info[cus_act_trd_info.gen=='Senior'].groupby('orr_mdi_dit_cd').count().reset_index()

labels = ["HTS", "MTS", "영업점", "유선"]
fig = make_subplots(1, 2, specs=[[{'type':'domain'}, {'type':'domain'}]],subplot_titles=['Y&Z세대', 'Senior'])
fig.add_trace(go.Pie(labels=labels, values=df1.cus_id,name="Y&Z세대"), 1, 1)
fig.add_trace(go.Pie(labels=labels, values=df2.cus_id,name="Senior"), 1, 2)
fig.update_layout(title={'text': "세대간 주문매체 비교",'x' : 0.5,})
fig.show()

대부분의 주식거래는 온라인으로 이루어지고 있습니다. 특히 Y&Z세대의 거래 93%는 휴대폰으로 이루어지고 있습니다. 따라서 Y&Z세대에 특화된 상품을 제공하기 위해서는 모바일 기기에서 구입할 수 있는 상품을 구상해야합니다. 

## 2. Insight & Suggestion
### (1) Insight : 평온해진 주식 시장과 혈기왕성한 Y&Z 세대

-큰 변동성으로 좋은 수익을 올릴 수 있었지만 vix가 잠잠해지는 추세임 즉 시장이 안정화되고 레버리지나 인버스에 투자해서 큰 수익을 올리기 힘들다. 
이제는 고전적인 분산투자를 해야한다. 하지만 해외투자를 이끌어가는 혈기왕성한 Y&Z투자자들을 위한 제안을 하지 하하하하

### (2) Suggestion : Robo Advisor의 Event Driven Investment with Sentiment Analysis Report



Event Driven Investment란 수익 콜, 파산, 합병, 인수 또는 분사 등과 같은 기업 이벤트 전후에 발생할 수 있는 가격 비효율을 활용하려는 투자 전략입니다.
전통적인 주식 투자자들은 이러한 기업 이벤트를 적절하게 분석하는 데 필요한 전문지식이 부족하고 정보에 접근할 수 없기 때문에 사모펀드나 헤지펀드가 많이 사용하는 전략입니다.<br>
하지만 게임, 영화, 공연 등 엔터테이먼트 산업의 경우 정보의 비대칭성이 상대적으로 적은 테마 분야입니다. 대부분의 스케쥴이 공개되기 때문입니다. 특히 대부분의 국외/국내 게임사들은 신규 게임 출시 예정 캘린더가 있습니다. 이를 통해 게임 출시예정일이 다가오는 경우 커뮤니티 사이트나 게임포탈에서 해당 게임에 대한 긍/부정 반응을 분석해 Event Driven Investment 전략을 사용할 수 있습니다.<br>
출시 전 게임사에 투자하고 출시이후 예측반응에 따라 주가상승이 일어나면 매도하는 전략과 출시 후 유저들의 반응을 분석하여 게임사에 투자하고 주가 상승이 일어나면 매도하는 전략을 취할 수 있습니다.<br>


### Nintendo사의 출시예정 게임 리스트
![coming_soon](./data/coming_soon.png)

이 때 고객의 긍/부정 반응을 분석하는데 사용할 수 있는 데이터 분석 기법은 감성분석(Sentiment Analysis)입니다. 자연어 분석(Natural Language Processing)의 한 분야인 감성분석이란 텍스트에 들어있는 의견이나 감성, 평가, 태도 등의 주관적인 정보를 분석하는 러닝 기법입니다. <br>
이를 통해 미국의 Reddit이나 Youtube, 일본의 5ch, 한국의 디시인사이드와 같은 대형 커뮤니티의 텍스트를 분석해 게임이나 영화에 대한 출시전 기대, 출시 후 반응 등을 분석할 수 있습니다. 이러한 커뮤니티의 경우 뉴스기사와 달리 이용자들의 생생한 리뷰 데이터를 확보할 수 있다는 장점이 있습니다. 또한 게임뿐만 아니라 거의 모든 산업에 대한 정보가 유통되기 때문에 테마에 국한되지 않는 정보를 얻을 수 있습니다. <br>
"데이터, 문화가 되다 : League1", AI야, 진짜 뉴스를 찾아줘! 에서는 진짜 뉴스와 가짜 뉴스를 구분하는 대회가 열리고 있는데 이 대회에서 사용되는 기법을 통해 홍보성 글이나 경쟁사의 악성 댓글과 같이 분석에 악영향을 주는 데이터를 가려낼 수 있을 것이라고 생각됩니다.<br> 

### (3) POC(Proof of Concept) : Nintendo
Nintendo사의 히트작 "모여봐요 동물의 숲"에 대해 감성분석을 진행하여 Suggestion에 대해 POC(Proof of concept)하고자 합니다.

### Nintendo사의 히트작 모여봐요 동물의 숲
![동숲](./data/동숲.jpg)

In [344]:
nintendo=pd.read_csv('./data/nintendo.csv')
fig = px.line(nintendo, x="Date", y="Close")
fig.update_layout(title={'text': "2019/2020 Nintendo 주가",'x' : 0.5,},xaxis_title="",yaxis_title="주가(엔)")
fig.update_xaxes(dtick="M1",tickformat="%b\n%Y",ticklabelmode="period")
#fig.add_shape(type="line",x0='2020-03-20', y0=0, x1='2020-03-20', y1=70000,line=dict(color="Red",width=1))
#text="모여봐요 동물의 숲 출시일"
#fig.add_annotation(x=0.45,y=0.9,text=text,xref="paper",yref="paper",font_size=20)
fig.add_annotation(x='2020-03-20',y=37810,xref="x",yref="y",axref='x',ayref='y',text="모여봐요 동물의 숲 출시일",showarrow=True,
        font=dict(family="Courier New, monospace",size=16,color="#636363"),
        align="center",arrowhead=2,arrowsize=1,arrowwidth=2,arrowcolor="#636363",ax='2020-9-22',ay=40000,
       # bordercolor="#c7c7c7",borderwidth=2,borderpad=4,
      #  bgcolor="#0effb3", opacity=0.8
        )
fig.show()


In [307]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px


font_location = 'C:/WINDOWS/fonts/HANBatang.ttf'
font_name = fm.FontProperties(fname=font_location).get_name()
plt.rc('font', family=font_name)

In [ ]:
act_info=pd.read_csv(r"2_act_info.csv")
cus_info=pd.read_csv(r"2_cus_info.csv")
iem_info=pd.read_csv(r"2_iem_info.csv")
trd_kr=pd.read_csv(r"2_trd_kr.csv")
trd_oss=pd.read_csv(r"2_trd_oss.csv")
krx_battery=pd.read_csv(r"KRX2차전지지수.csv",encoding='euc-kr')
krx_bbig=pd.read_csv(r"KRXBBIG지수.csv",encoding='euc-kr')
krx_bio=pd.read_csv(r"KRXBIO지수.csv",encoding='euc-kr')
krx_eco=pd.read_csv(r"KRXECO지수.csv",encoding='euc-kr')
krx_esgs=pd.read_csv(r"KRXESG사회책임지수.csv",encoding='euc-kr')
krx_esg=pd.read_csv(r"KRXESG지수.csv",encoding='euc-kr')
krx_game=pd.read_csv(r"KRX게임지수.csv",encoding='euc-kr')
krx_internet=pd.read_csv(r"KRX인터넷지수.csv",encoding='euc-kr')


전처리